# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 3.2.2
- **Date :** 10/11/2023
- **Heure :** 22:34
- **Données :** `ready_Evaporation_Sunshine_Cloud9am_Cloud3pm.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** TSF

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../../data/processed/omar/ready_Evaporation_Sunshine_Cloud9am_Cloud3pm.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,year,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,13.1,24.2,0.0,61.0,22.0,33.0,36.0,34.0,1005.6,1003.6,...,2008,12,1,16,2.748894,2.356194,2.748894,-35.420977,149.092134,1
2008-12-01,17.0,30.4,0.0,31.0,17.0,22.0,52.0,61.0,1013.5,1010.7,...,2008,12,1,30,1.963495,1.963495,0.785398,-28.002373,153.414599,3
2008-12-02,5.7,24.2,0.0,48.0,17.0,26.0,38.0,25.0,1008.8,1006.3,...,2008,12,2,16,2.748894,1.963495,2.748894,-35.420977,149.092134,1
2008-12-02,19.0,29.7,0.0,37.0,19.0,31.0,55.0,70.0,1013.1,1010.2,...,2008,12,2,30,1.178097,1.963495,1.178097,-28.002373,153.414599,3
2008-12-03,9.4,25.7,0.0,50.0,11.0,24.0,35.0,26.0,1005.7,1005.9,...,2008,12,3,16,2.748894,1.963495,2.356194,-35.420977,149.092134,1


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18907 entries, 2008-12-01 to 2017-06-25
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MinTemp         18907 non-null  float64
 1   MaxTemp         18907 non-null  float64
 2   Rainfall        18907 non-null  float64
 3   WindGustSpeed   18907 non-null  float64
 4   WindSpeed9am    18907 non-null  float64
 5   WindSpeed3pm    18907 non-null  float64
 6   Humidity9am     18907 non-null  float64
 7   Humidity3pm     18907 non-null  float64
 8   Pressure9am     18907 non-null  float64
 9   Pressure3pm     18907 non-null  float64
 10  Temp9am         18907 non-null  float64
 11  Temp3pm         18907 non-null  float64
 12  RainToday       18907 non-null  int64  
 13  RainTomorrow    18907 non-null  int64  
 14  year            18907 non-null  int64  
 15  month           18907 non-null  int64  
 16  day             18907 non-null  int64  
 17  LocationNum     18907 

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [8]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 4. Modélisation

In [9]:
# Instanciation d'un modèle classificateur via un Pipeline

from sklearn.pipeline import Pipeline
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.interval_based import TimeSeriesForestClassifier

steps = [
    ("concatenate", ColumnConcatenator()), ## Cette étape permet de transformer les jeux de données multivariées en univariées afin qu'ils puissent être traités par `TimeSeriesForestClassifier`.
    ("classify", TimeSeriesForestClassifier()),
]

clf_rf_ts = Pipeline(steps)

In [10]:
# Récupération des paramètres initiaux de la TSF

clf_rf_ts["classify"].get_params()

{'min_interval': 3, 'n_estimators': 200, 'n_jobs': 1, 'random_state': None}

In [11]:
# Entraînement du modèle

clf_rf_ts.fit(X_train, y_train)

Pipeline(steps=[('concatenate', ColumnConcatenator()),
                ('classify', TimeSeriesForestClassifier())])

In [12]:
# Récupération des paramètres ajustés de la TSF

clf_rf_ts["classify"].get_fitted_params()

{'classes': array([0, 1]),
 'intervals': [array([[ 6,  9],
         [ 0, 20],
         [13, 19],
         [ 1, 14]]),
  array([[ 7, 18],
         [15, 21],
         [ 2, 18],
         [ 0, 19]]),
  array([[ 1, 11],
         [ 9, 13],
         [ 3, 11],
         [ 8, 18]]),
  array([[19, 22],
         [ 4, 21],
         [16, 19],
         [13, 16]]),
  array([[ 5,  8],
         [12, 17],
         [ 6, 14],
         [ 8, 16]]),
  array([[ 2, 14],
         [ 0,  6],
         [ 2,  9],
         [13, 16]]),
  array([[13, 16],
         [ 9, 12],
         [ 9, 12],
         [ 9, 12]]),
  array([[ 7, 11],
         [ 5, 13],
         [19, 22],
         [ 8, 19]]),
  array([[17, 20],
         [19, 22],
         [15, 21],
         [ 3, 12]]),
  array([[15, 18],
         [ 3,  6],
         [16, 21],
         [19, 22]]),
  array([[ 3, 14],
         [ 4, 20],
         [ 1, 19],
         [14, 17]]),
  array([[13, 20],
         [ 7, 21],
         [14, 17],
         [14, 20]]),
  array([[14, 17],
     

In [13]:
# Réalisation des prédictions

y_pred = clf_rf_ts.predict(X_test)

## 5. Évaluation

In [14]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,2227,119
1,403,402


In [15]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90      2346
           1       0.77      0.50      0.61       805

    accuracy                           0.83      3151
   macro avg       0.81      0.72      0.75      3151
weighted avg       0.83      0.83      0.82      3151

